# Esercizio 1 
#### Input e download 

In [1]:
import nltk
import random
from nltk.corpus import movie_reviews
from nltk.corpus import stopwords, words
from nltk.corpus.util import LazyCorpusLoader
from nltk.corpus.reader import *
from nltk.stem import WordNetLemmatizer
import re
from nltk.metrics.scores import precision, recall, f_measure
import numpy as np
from collections import defaultdict
from nltk.tokenize import word_tokenize
#nltk.download('europarl_raw')
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('omw-1.4')
#nltk.download('punkt')

In [20]:
def find_features(document):
    words = set(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)
    return features

### Importo il  raw  di EuroparlCorpusReader da diverse lingue 
Salvo **40000** parole in lingua **inglese** e **10000** parole di **ogni altra lingua importata**

In [4]:
english = LazyCorpusLoader('europarl_raw/english', EuroparlCorpusReader, r'ep-.*\.en', encoding='utf-8')
italian =  LazyCorpusLoader('europarl_raw/italian', EuroparlCorpusReader, r'ep-.*\.it', encoding='utf-8')
french =   LazyCorpusLoader('europarl_raw/french', EuroparlCorpusReader, r'ep-.*\.fr', encoding='utf-8')
spanish = LazyCorpusLoader('europarl_raw/spanish', EuroparlCorpusReader, r'ep-.*\.es', encoding='utf-8')
german = LazyCorpusLoader('europarl_raw/german', EuroparlCorpusReader, r'ep-.*\.de', encoding='utf-8')

words = english.words()[:40000]
words += italian.words()[:10000]
words += french.words()[:10000]
words += spanish.words()[:10000]
words += german.words()[:10000]


Salvo tutte le stop words cioè le parole più "coomuni" delle varie lingue importate 

In [5]:
stop_words_list = set(stopwords.words('english'))
stop_words_list.union(set(stopwords.words('italian')))
stop_words_list.union(set(stopwords.words('french')))
stop_words_list.union(set(stopwords.words('spanish')))
stop_words_list.union(set(stopwords.words('german')))

{'a',
 'aber',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'alle',
 'allem',
 'allen',
 'aller',
 'alles',
 'als',
 'also',
 'am',
 'an',
 'and',
 'ander',
 'andere',
 'anderem',
 'anderen',
 'anderer',
 'anderes',
 'anderm',
 'andern',
 'anderr',
 'anders',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'auch',
 'auf',
 'aus',
 'be',
 'because',
 'been',
 'before',
 'bei',
 'being',
 'below',
 'between',
 'bin',
 'bis',
 'bist',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'da',
 'damit',
 'dann',
 'das',
 'dass',
 'dasselbe',
 'dazu',
 'daß',
 'dein',
 'deine',
 'deinem',
 'deinen',
 'deiner',
 'deines',
 'dem',
 'demselben',
 'den',
 'denn',
 'denselben',
 'der',
 'derer',
 'derselbe',
 'derselben',
 'des',
 'desselben',
 'dessen',
 'dich',
 'did',
 'didn',
 "didn't",
 'die',
 'dies',
 'diese',
 'dieselbe',
 'dieselben',
 'diesem',
 'diesen',
 'dieser',
 'dieses',
 'dir',
 'do',
 'doch',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',


Uso i vari tipi di "filtri" per elliminare parole o ridurre alla "radice" le parole per poi andare a eseguire uno shuffle della lista

In [6]:
lemmatizer = WordNetLemmatizer()
filtered_sentence = []
for w in words:
    w = lemmatizer.lemmatize(w)
    if w not in stop_words_list :
        filtered_sentence.append(w)
np.random.shuffle(filtered_sentence)


Calculate the **frequency distribution** of the words in the corpus

In [7]:
word_frequency_distribution = nltk.FreqDist([word.lower() for word in filtered_sentence])

Prendo le prime 8000 parole -> **? più utilizate ?**

In [8]:
#sorted = [i[0] for i in word_frequency_distribution.most_common(8000)]
word_features = list(word_frequency_distribution.keys())[:8000]


Creo il featuresets prendendo 8000 sentence per la lingua inglese e 1000 per le altre

In [9]:
featuresets = [ (find_features(sentence),"eng" ) for sentence in english.sents()[:8000]]
featuresets += [ (find_features(sentence),"not eng" ) for sentence in italian.sents()[:1000]]
featuresets += [ (find_features(sentence),"not eng" ) for sentence in french.sents()[:1000]]
featuresets += [ (find_features(sentence),"not eng" ) for sentence in spanish.sents()[:1000]]
featuresets += [ (find_features(sentence),"not eng" ) for sentence in german.sents()[:1000]]
np.random.shuffle(featuresets)



Split the featuresets and obtain training and testing

In [10]:
training_set = featuresets[0:int(len(featuresets) * 0.8)]
testing_set = featuresets[int(len(featuresets) * 0.8):]


I train my classifier with training_set

In [11]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

## Make all the metric
#### Accuracy:

In [12]:
print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, testing_set))*100)

Classifier accuracy percent: 99.70833333333333


#### Precision, Recall:

In [13]:
predictions, gold_labels = defaultdict(set), defaultdict(set)

for i, (features, label) in enumerate(testing_set):
    predictions[classifier.classify(features)].add(i)
    gold_labels[label].add(i) 

for label in predictions:
    print(label, 'Precision:', precision(gold_labels[label], predictions[label]))
    print(label, 'Recall:', recall(gold_labels[label], predictions[label]))
    print()

eng Precision: 0.9955357142857143
eng Recall: 1.0
eng F1-Score: 0.9977628635346755

not eng Precision: 1.0
not eng Recall: 0.9916567342073898
not eng F1-Score: 0.9958108916816277



### Confusion Matrix

In [14]:
test_result = []
gold_result = []

for i in range(len(testing_set)):
    test_result.append(classifier.classify(testing_set[i][0]))
    gold_result.append(testing_set[i][1])
CM = nltk.ConfusionMatrix(gold_result, test_result)
print(CM)

        |         n |
        |         o |
        |         t |
        |           |
        |    e    e |
        |    n    n |
        |    g    g |
--------+-----------+
    eng |<1561>   . |
not eng |    7 <832>|
--------+-----------+
(row = reference; col = test)



## Show most informative features of classifier

In [33]:
classifier.show_most_informative_features(10)

Most Informative Features
                      en = True           not en : eng    =    536.5 : 1.0
                     die = True           not en : eng    =    530.2 : 1.0
                      et = True           not en : eng    =    501.7 : 1.0
                      el = True           not en : eng    =    489.5 : 1.0
                      se = True           not en : eng    =    436.5 : 1.0
                      le = True           not en : eng    =    415.9 : 1.0
                     den = True           not en : eng    =    262.7 : 1.0
                       e = True           not en : eng    =    226.7 : 1.0
                       l = True           not en : eng    =    212.5 : 1.0
                      je = True           not en : eng    =    173.1 : 1.0


## Test the classifier

In [25]:
def test(text):
    text= word_tokenize(text)
    feat = find_features(text)
    return classifier.classify(feat)

### Insert a string and return if it is or it is not Eng

In [32]:

print(test("Hola amigo como estas"))

not eng
